# 문자 단위 RNN



In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [2]:
input_str = 'apple'
label_str = 'pple!'
char_vocab = sorted(list(set(input_str+label_str)))
vocab_size = len(char_vocab)
print ('문자 집합의 크기 : {}'.format(vocab_size))

문자 집합의 크기 : 5


In [3]:
input_size = vocab_size # 입력의 크기는 문자 집합의 크기
hidden_size = 5
output_size = 5
learning_rate = 0.1

In [4]:
char_to_index = dict((c, i) for i, c in enumerate(char_vocab)) # 문자에 고유한 정수 인덱스 부여
print(char_to_index)

{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}


In [5]:
index_to_char={}
for key, value in char_to_index.items():
    index_to_char[value] = key
print(index_to_char)

{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}


In [6]:
x_data = [char_to_index[c] for c in input_str]
y_data = [char_to_index[c] for c in label_str]
print(x_data)
print(y_data)

[1, 4, 4, 3, 2]
[4, 4, 3, 2, 0]


In [7]:
# 배치 차원 추가
# 텐서 연산인 unsqueeze(0)를 통해 해결할 수도 있었음.
x_data = [x_data]
y_data = [y_data]
print(x_data)
print(y_data)

[[1, 4, 4, 3, 2]]
[[4, 4, 3, 2, 0]]


In [8]:
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]
print(x_one_hot)

[array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])]


In [9]:
np.eye(5)

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

In [10]:
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

<ipython-input-10-c1bfbd518a63>:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  X = torch.FloatTensor(x_one_hot)


In [11]:
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([1, 5, 5])
레이블의 크기 : torch.Size([1, 5])


In [12]:
class Net(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True) # RNN 셀 구현
        self.fc = torch.nn.Linear(hidden_size, output_size, bias=True) # 출력층 구현

    def forward(self, x): # 구현한 RNN 셀과 출력층을 연결
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [13]:
net = Net(input_size, hidden_size, output_size)

In [14]:
outputs = net(X)
print(outputs.shape) # 3차원 텐서

torch.Size([1, 5, 5])


In [16]:
print(outputs.view(-1, input_size).shape) # 2차원 텐서로 변환


torch.Size([5, 5])


In [17]:
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([1, 5])
torch.Size([5])


In [18]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [19]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, input_size), Y.view(-1)) # view를 하는 이유는 Batch 차원 제거를 위해
    loss.backward() # 기울기 계산
    optimizer.step() # 아까 optimizer 선언 시 넣어둔 파라미터 업데이트

    # 아래 세 줄은 모델이 실제 어떻게 예측했는지를 확인하기 위한 코드.
    result = outputs.data.numpy().argmax(axis=2) # 최종 예측값인 각 time-step 별 5차원 벡터에 대해서 가장 높은 값의 인덱스를 선택
    result_str = ''.join([index_to_char[c] for c in np.squeeze(result)])
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)

0 loss:  1.5847843885421753 prediction:  [[2 2 2 2 2]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  eeeee
1 loss:  1.376948595046997 prediction:  [[4 4 4 4 4]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  ppppp
2 loss:  1.2170974016189575 prediction:  [[4 4 4 2 4]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pppep
3 loss:  1.0138742923736572 prediction:  [[4 4 4 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pppe!
4 loss:  0.8117397427558899 prediction:  [[4 4 4 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pppe!
5 loss:  0.6145950555801392 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
6 loss:  0.4474720358848572 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
7 loss:  0.34007176756858826 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
8 loss:  0.24937565624713898 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
9 loss:  0.17458918690681458 prediction:  [[4 4 3 2 0]

더 많은 데이터 활용

In [20]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

In [21]:
sentence

"if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea."

In [22]:
char_set = list(set(sentence)) # 중복을 제거한 문자 집합 생성
char_dic = {c: i for i, c in enumerate(char_set)} # 각 문자에 정수 인코딩

In [24]:
char_dic

{'b': 0,
 '.': 1,
 'y': 2,
 'w': 3,
 'i': 4,
 'h': 5,
 'd': 6,
 'm': 7,
 'f': 8,
 'e': 9,
 'c': 10,
 'g': 11,
 ' ': 12,
 'k': 13,
 'n': 14,
 'o': 15,
 ',': 16,
 'p': 17,
 'r': 18,
 'a': 19,
 't': 20,
 'l': 21,
 "'": 22,
 's': 23,
 'u': 24}

In [25]:
dic_size = len(char_dic)
print('문자 집합의 크기 : {}'.format(dic_size))

문자 집합의 크기 : 25


In [26]:
# 하이퍼파라미터 설정
hidden_size = dic_size
sequence_length = 10  # 임의 숫자 지정
learning_rate = 0.1

In [27]:
# 데이터 구성
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    print(i, x_str, '->', y_str)

    x_data.append([char_dic[c] for c in x_str])  # x str to index
    y_data.append([char_dic[c] for c in y_str])  # y str to index

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [29]:
x_data

[[4, 8, 12, 2, 15, 24, 12, 3, 19, 14],
 [8, 12, 2, 15, 24, 12, 3, 19, 14, 20],
 [12, 2, 15, 24, 12, 3, 19, 14, 20, 12],
 [2, 15, 24, 12, 3, 19, 14, 20, 12, 20],
 [15, 24, 12, 3, 19, 14, 20, 12, 20, 15],
 [24, 12, 3, 19, 14, 20, 12, 20, 15, 12],
 [12, 3, 19, 14, 20, 12, 20, 15, 12, 0],
 [3, 19, 14, 20, 12, 20, 15, 12, 0, 24],
 [19, 14, 20, 12, 20, 15, 12, 0, 24, 4],
 [14, 20, 12, 20, 15, 12, 0, 24, 4, 21],
 [20, 12, 20, 15, 12, 0, 24, 4, 21, 6],
 [12, 20, 15, 12, 0, 24, 4, 21, 6, 12],
 [20, 15, 12, 0, 24, 4, 21, 6, 12, 19],
 [15, 12, 0, 24, 4, 21, 6, 12, 19, 12],
 [12, 0, 24, 4, 21, 6, 12, 19, 12, 23],
 [0, 24, 4, 21, 6, 12, 19, 12, 23, 5],
 [24, 4, 21, 6, 12, 19, 12, 23, 5, 4],
 [4, 21, 6, 12, 19, 12, 23, 5, 4, 17],
 [21, 6, 12, 19, 12, 23, 5, 4, 17, 16],
 [6, 12, 19, 12, 23, 5, 4, 17, 16, 12],
 [12, 19, 12, 23, 5, 4, 17, 16, 12, 6],
 [19, 12, 23, 5, 4, 17, 16, 12, 6, 15],
 [12, 23, 5, 4, 17, 16, 12, 6, 15, 14],
 [23, 5, 4, 17, 16, 12, 6, 15, 14, 22],
 [5, 4, 17, 16, 12, 6, 15, 14, 22,

In [30]:
x_one_hot = [np.eye(dic_size)[x] for x in x_data] # x 데이터는 원-핫 인코딩
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [32]:
len(x_one_hot)

170

In [33]:
x_one_hot[0]

array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [34]:
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([170, 10, 25])
레이블의 크기 : torch.Size([170, 10])


In [35]:
Y[0]

tensor([ 8, 12,  2, 15, 24, 12,  3, 19, 14, 20])

In [36]:
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers): # 현재 hidden_size는 dic_size와 같음.
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [37]:
net = Net(dic_size, hidden_size, 2) # 이번에는 층을 두 개 쌓습니다.


In [38]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [39]:
outputs = net(X)
print(outputs.shape) # 3차원 텐서

torch.Size([170, 10, 25])


In [40]:
print(outputs.view(-1, dic_size).shape) # 2차원 텐서로 변환.


torch.Size([1700, 25])


In [41]:
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([170, 10])
torch.Size([1700])


In [42]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X) # (170, 10, 25) 크기를 가진 텐서를 매 에포크마다 모델의 입력으로 사용
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    # results의 텐서 크기는 (170, 10)
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오지만
            predict_str += ''.join([char_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += char_set[result[-1]]

    print(predict_str)

''shl'isssaihi's'hihihik'hiaihi'saihipl'hlspli'a'ihi'siiiiihi'sliiihiiplshihliihi'saihlkiaaia'il'hl'i'ihliisl's'ipslipll'iiihihs'ihiil'hi'hi's'hs'shiiiiliiiiiihls'''iiiihssh'iikih
                                                                                                                                                                                   
taaoaoaoaoatoaoaoaotthaoaoaoaoaoaoaoaoaottatoaoaotoaoaoaoaottoataetthaoaoaotehaoaoaoaoaotoaoteoattehthaoaoaoaoaoaoaoaoaoaoaoathaeaoaotttaoetttaoaoaoaoaoaoaoaoaoahaoetteotoaoaoaoao
  t  t  t  ttttt t  t  t t   t t  t  t  tt t  t rt t  t   t  tt    t  ttt     t tt tt  t   t tt  t  t t  tt  t t t ttt ttt t t t t  t  t  t t  t tt t  tt  t   t t     tt     t  t 
    o  o            e      o   n o   n          e         e     o  r  o   o                           o  e                       e  n    en e  e   o   o     o     o          o  n 
      ie,i' ''i'ii'' l'i 'i 'i i' i'' ''i'i ' '' ''ii''i ' l'il' '' ''o'' oi''i'''l il'o''i'l i'oe'i

# 단어 단위 RNN

In [43]:
import torch
import torch.nn as nn
import torch.optim as optim

In [44]:
sentence = "Repeat is the best medicine for memory".split()


In [45]:
vocab = list(set(sentence))
print(vocab)

['for', 'Repeat', 'is', 'medicine', 'memory', 'best', 'the']


In [46]:
word2index = {tkn: i for i, tkn in enumerate(vocab, 1)}  # 단어에 고유한 정수 부여
word2index['<unk>']=0

In [47]:
print(word2index)

{'for': 1, 'Repeat': 2, 'is': 3, 'medicine': 4, 'memory': 5, 'best': 6, 'the': 7, '<unk>': 0}


In [48]:
print(word2index['memory'])


5


In [49]:
# 수치화된 데이터를 단어로 바꾸기 위한 사전
index2word = {v: k for k, v in word2index.items()}
print(index2word)

{1: 'for', 2: 'Repeat', 3: 'is', 4: 'medicine', 5: 'memory', 6: 'best', 7: 'the', 0: '<unk>'}


In [50]:
def build_data(sentence, word2index):
    encoded = [word2index[token] for token in sentence] # 각 문자를 정수로 변환. 
    input_seq, label_seq = encoded[:-1], encoded[1:] # 입력 시퀀스와 레이블 시퀀스를 분리
    input_seq = torch.LongTensor(input_seq).unsqueeze(0) # 배치 차원 추가
    label_seq = torch.LongTensor(label_seq).unsqueeze(0) # 배치 차원 추가
    return input_seq, label_seq

In [51]:
X, Y = build_data(sentence, word2index)

In [52]:
print(X)
print(Y)

tensor([[2, 3, 7, 6, 4, 1]])
tensor([[3, 7, 6, 4, 1, 5]])


In [53]:
class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super(Net, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, # 워드 임베딩
                                            embedding_dim=input_size)
        self.rnn_layer = nn.RNN(input_size, hidden_size, # 입력 차원, 은닉 상태의 크기 정의
                                batch_first=batch_first)
        self.linear = nn.Linear(hidden_size, vocab_size) # 출력은 원-핫 벡터의 크기를 가져야함. 또는 단어 집합의 크기만큼 가져야함.

    def forward(self, x):
        # 1. 임베딩 층
        # 크기변화: (배치 크기, 시퀀스 길이) => (배치 크기, 시퀀스 길이, 임베딩 차원)
        output = self.embedding_layer(x)
        # 2. RNN 층
        # 크기변화: (배치 크기, 시퀀스 길이, 임베딩 차원)
        # => output (배치 크기, 시퀀스 길이, 은닉층 크기), hidden (1, 배치 크기, 은닉층 크기)
        output, hidden = self.rnn_layer(output)
        # 3. 최종 출력층
        # 크기변화: (배치 크기, 시퀀스 길이, 은닉층 크기) => (배치 크기, 시퀀스 길이, 단어장 크기)
        output = self.linear(output)
        # 4. view를 통해서 배치 차원 제거
        # 크기변화: (배치 크기, 시퀀스 길이, 단어장 크기) => (배치 크기*시퀀스 길이, 단어장 크기)
        return output.view(-1, output.size(2))

In [57]:
vocab_size

8

In [54]:
# 하이퍼 파라미터
vocab_size = len(word2index)  # 단어장의 크기는 임베딩 층, 최종 출력층에 사용된다. <unk> 토큰을 크기에 포함한다.
input_size = 5  # 임베딩 된 차원의 크기 및 RNN 층 입력 차원의 크기
hidden_size = 20  # RNN의 은닉층 크기

In [60]:
# 모델 생성
model = Net(vocab_size, input_size, hidden_size, batch_first=True)
# 손실함수 정의
loss_function = nn.CrossEntropyLoss() # 소프트맥스 함수 포함이며 실제값은 원-핫 인코딩 안 해도 됨.
# 옵티마이저 정의
optimizer = optim.Adam(params=model.parameters())

In [56]:
# 임의로 예측해보기. 가중치는 전부 랜덤 초기화 된 상태이다.
output = model(X)
print(output)

tensor([[ 0.2396, -0.2473, -0.2938, -0.3068,  0.5220, -0.0239, -0.2340,  0.0285],
        [-0.3767, -0.1616, -0.3012, -0.0468, -0.1227,  0.3465,  0.2663,  0.0856],
        [-0.3933,  0.1195, -0.0882,  0.2058,  0.1384,  0.3180, -0.0703, -0.0584],
        [-0.3772, -0.0769, -0.2327,  0.2912,  0.4598, -0.1309, -0.0575,  0.2115],
        [-0.3359,  0.0888, -0.1719,  0.3178,  0.1754, -0.0426,  0.0904, -0.0899],
        [-0.6043,  0.0930, -0.0345,  0.2958, -0.0339,  0.0264,  0.2607, -0.1717]],
       grad_fn=<ViewBackward0>)


In [63]:
output.softmax(-1).argmax(-1)

tensor([3, 7, 6, 4, 1, 5])

In [65]:
decode(output.softmax(-1).argmax(-1).tolist())

['is', 'the', 'best', 'medicine', 'for', 'memory']

In [58]:
decode = lambda y: [index2word.get(x) for x in y]


In [61]:
# 훈련 시작
for step in range(201):
    # 경사 초기화
    optimizer.zero_grad()
    # 순방향 전파
    output = model(X)
    # 손실값 계산
    loss = loss_function(output, Y.view(-1))
    # 역방향 전파
    loss.backward()
    # 매개변수 업데이트
    optimizer.step()
    # 기록
    if step % 40 == 0:
        print("[{:02d}/201] {:.4f} ".format(step+1, loss))
        pred = output.softmax(-1).argmax(-1).tolist()
        print(decode(pred))
        print(" ".join(["Repeat"] + decode(pred)))
        print()

[01/201] 2.1226 
['for', 'the', 'the', 'memory', 'memory', 'memory']
Repeat for the the memory memory memory

[41/201] 1.5330 
['is', 'the', 'best', 'memory', 'memory', 'memory']
Repeat is the best memory memory memory

[81/201] 0.9428 
['is', 'the', 'best', 'medicine', 'for', 'memory']
Repeat is the best medicine for memory

[121/201] 0.5143 
['is', 'the', 'best', 'medicine', 'for', 'memory']
Repeat is the best medicine for memory

[161/201] 0.2801 
['is', 'the', 'best', 'medicine', 'for', 'memory']
Repeat is the best medicine for memory

[201/201] 0.1608 
['is', 'the', 'best', 'medicine', 'for', 'memory']
Repeat is the best medicine for memory

